In [1]:
import pandas as pd
import math
import pickle

In [2]:
data = pd.read_excel('IR_Spring2021_ph12_7k.xlsx')

In [3]:
#class below is for holding single id and single url
class holder:
    def __init__(self,ID = 0,url = None):
        self.ID = ID
        self.url = url
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url


In [4]:
#class below is for holding term frequency and list of holder classess
class termValue:
    def __init__(self):
        self.frequency = 0
        self.holderList = []
    def addHolder(self,newHolder):
        if newHolder not in self.holderList:
            self.holderList.append(newHolder)

In [5]:
#function below is for calculating frequency of each term
def frequencyCalc(myDictionary):
    for i in myDictionary:
        myDictionary[i].frequency = len(myDictionary[i].holderList)

In [6]:
"""
#algorithm below is for retrieval of the dictionary
myFile = open('dictionary.txt',encoding = 'UTF-8')
theDictionary = {}
for i in myFile:
    res = i.split('$')
    if res[0] not in theDictionary:
        theDictionary[res[0]] = termValue()
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
    else:
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
myFile.close()
"""

"\n#algorithm below is for retrieval of the dictionary\nmyFile = open('dictionary.txt',encoding = 'UTF-8')\ntheDictionary = {}\nfor i in myFile:\n    res = i.split('$')\n    if res[0] not in theDictionary:\n        theDictionary[res[0]] = termValue()\n        myHolder = holder(res[1],res[2])\n        theDictionary[res[0]].addHolder(myHolder)\n    else:\n        myHolder = holder(res[1],res[2])\n        theDictionary[res[0]].addHolder(myHolder)\nmyFile.close()\n"

In [7]:
"""
#algorithm above was inefficient it took about 5 minutes so saving and retrival using pickle is better
pickle.dump(theDictionary,open('dictionary','ab'))
"""

"\n#algorithm above was inefficient it took about 5 minutes so saving and retrival using pickle is better\npickle.dump(theDictionary,open('dictionary','ab'))\n"

In [8]:
#code below is for retrival our dictionary
theDictionary = pickle.load(open('dictionary','rb'))

In [9]:
frequencyCalc(theDictionary)

In [10]:
#list below is just for saving list of words
wordsList = []
for i in theDictionary:
    wordsList.append(i)

In [11]:
#normalize 1 method
def normalize1(myList):
    """"
    we should remove some specific characters eg: we do not save dot in the end of sentence with the last word
    we just need to remove if these specific chars are at the last of our word
    """
    for i in range(len(myList)):
        if myList[i].endswith('.'):
            myList[i] = myList[i].replace('.','')
        elif myList[i].endswith('،'):
            myList[i] = myList[i].replace('،','')
        elif myList[i].endswith(':'):
            myList[i] = myList[i].replace(':','')
        else:
            pass
    #the two last members means end of message and it is in all docs so we do not need it
    myList.pop()
    myList.pop()

In [12]:
#algorithm below is saving documents and their bag of words in dictionary docid as key and list of words as value
bagOfWordsDic = {}
for i in range(len(data)):
    docId = data['id'][i]
    listOfwords = data['content'][i].split()
    normalize1(listOfwords)
    bagOfWordsDic[docId] = wordsList

In [26]:
#function below is for calculating tf
def tf_calculator(term,document):
    documentBagOfWords = document.split()
    normalize1(documentBagOfWords)
    n = documentBagOfWords.count(term)
    if n == 0:
        return 0
    return 1+math.log(n)

In [27]:
#function below is for calculating tf for a query
def tf_calculatorV02(term,query):
    queryBagOfWords = query.split()
    n = queryBagOfWords.count(term)
    if n == 0:
        return 0
    return 1+math.log(n)

In [28]:
#function below is for calculating idf
def idf_calculator(term,N,myDictionary):
    return math.log(N/myDictionary[term].frequency)

In [29]:
#function below is for calculating vector of a single document
def createVector(document,myDictionary,N):
    documentBagOfWords = document.split()
    normalize1(documentBagOfWords)
    resDict = {}
    for i in documentBagOfWords:
        resDict[i] = tf_calculator(i,document)*idf_calculator(i,N,myDictionary)
    return resDict

In [30]:
#function below is for calculating vector for a query
def createVectorV02(query,myDictionary,N):
    queryBagOfWords = query.split()
    resDict = {}
    for i in queryBagOfWords:
        resDict[i] = tf_calculatorV02(i,query)
    return resDict

In [31]:
"""
#algorithm below is for calculating all vector spaces
vectorDictionary = {}
for i in range(len(data)):
    documentId = data['id'][i]
    documentUrl = data['url'][i]
    documentContent = data['content'][i]
    vectorDictionary[str(documentId)+' '+documentUrl] = createVector(documentContent,theDictionary,len(data))
"""

"\n#algorithm below is for calculating all vector spaces\nvectorDictionary = {}\nfor i in range(len(data)):\n    documentId = data['id'][i]\n    documentUrl = data['url'][i]\n    documentContent = data['content'][i]\n    vectorDictionary[str(documentId)+' '+documentUrl] = createVector(documentContent,theDictionary,len(data))\n"

In [32]:
#saving the vector dictionary
#pickle.dump(vectorDictionary,open('vectorDictionary','ab'))

In [33]:
#loading vector dictionary
vectorDictionary = pickle.load(open('vectorDictionary','rb'))

In [34]:
#function below is for calculating size
def size_calculate(vector):
    res = 0
    for i in vector:
        res += math.pow(vector[i],2)
    return math.sqrt(res)

In [35]:
#class below is for saving vectorid + similarity to a query
class vectorSimilarity:
    def __init__(self,similarityValue,vectorId):
        self.similarityValue = similarityValue
        self.vectorId = vectorId
    def __str__(self):
        return self.vectorId
    def __lt__(self,obj):
        return self.similarityValue < obj.similarityValue

In [36]:
#calculating similarity between a document and query
def cosineSimilarity(query,ID,URL,documentVectors,N):
    queryVector = createVectorV02(query,theDictionary,N)
    documentVector = documentVectors[str(ID)+' '+URL]
    res = 0
    for i in queryVector:
        if i in documentVector:
            res += queryVector[i]*documentVector[i]
    return res/size_calculate(documentVector)

In [37]:
def engine():
    query = input('enter query: ')
    resList = []
    for i in range(len(data)):
        similarity = cosineSimilarity(query,data['id'][i],data['url'][i],vectorDictionary,len(data))
        resList.append(vectorSimilarity(similarity,str(data['id'][i])+' '+data['url'][i],))
    resList = sorted(resList,reverse = True)
    for i in range(10):
        print(resList[i])

In [39]:
engine()

enter query: تیم سپاهان فرصت
811 https://www.isna.ir/news/99101208771/برتری-تیم-فولاد-مبارکه-سپاهان-در-مرحله-مقدماتی-لیگ-برتر-والیبال
1139 https://www.isna.ir/news/98030401572/شکست-نزدیک-فوتبال-دانشجویی-ایران-برابر-شاگردان-قلعه-نویی
1655 https://www.isna.ir/news/98120504157/شکایت-سپاهان-از-مدیرعامل-پرسپولیس
767 https://www.isna.ir/news/99092116183/سرمربی-سپاهان-به-دنبال-نمایش-فوتبال-کلاسیک-هستیم
96 https://www.isna.ir/news/99022316860/عطش-مان-برای-فوتبال-بیش-از-دیگران-است
1024 https://www.isna.ir/news/99122317556/کاپیتان-تیم-ملی-سپاهانی-شد
48 https://www.isna.ir/news/99012011411/تعطیلی-لیگ-هندبال-تصمیم-درستی-است
1622 https://www.isna.ir/news/98112216716/امانوئل-سپاهان-تیم-بسیار-تهاجمی-است
1480 https://www.isna.ir/news/98090805350/حضور-فتحی-در-اردوی-استقلال-پیش-از-بازی-با-سپاهان-آبی-ها-پاداش
730 https://www.isna.ir/news/99091209903/تساوی-سایپا-و-تیم-ملی-جوانان-در-دیداری-دوستانه
